In [95]:
import pandas as pd
import numpy as np
import re
import joblib


# Đọc dữ liệu từ file CSV
file_path = '../data/lyric.csv'
df = pd.read_csv(file_path)

# Xác định cột chứa lời bài hát
lyrics_column = 'lyric'  # Thay bằng tên cột cụ thể nếu khác

# Load the Vietnamese stopwords
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines())

def clean_text(text):
    # 1. Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', ' ', text)  # Thay thế ký tự đặc biệt bằng khoảng trắng
    text = re.sub(r'\d+', ' ', text)  # Loại bỏ chữ số
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa

    # 2. Chuyển về chữ thường
    text = text.lower()

    # 3. Loại bỏ stopword
    if isinstance(text, str):  # Ensure text is a string
        words = text.split()
        cleaned_text = ' '.join([word for word in words if word not in stop_words])
        return cleaned_text
    return ' '.join(words)

# Áp dụng hàm xử lý lên cột chứa lời bài hát
df[lyrics_column] = df[lyrics_column].astype(str).apply(clean_text)
df.columns = ['index','name_song','lyric','label']
df.drop(['index'], axis=1,inplace=True)

processed_path = '../data/processed_data.csv'
df.to_csv(processed_path, index = False)


In [145]:
import numpy as np
import pandas as pd

def stratified_train_val_test_split(data, label_col, test_size=0.15, val_size=0.15, random_state=None):
        # Kiểm tra sự tồn tại của cột nhãn
        if label_col not in data.columns:
            raise ValueError(f"Cột '{label_col}' không tồn tại trong dữ liệu.")
        
        # Thiết lập giá trị random seed nếu cần
        if random_state is not None:
            np.random.seed(random_state)
        
        # Tạo danh sách các chỉ số cho từng tập
        train_indices = []
        val_indices = []
        test_indices = []
        
        # Duyệt qua từng nhãn để chia dữ liệu theo tỷ lệ
        for label in data[label_col].unique():
            # Lấy các chỉ số của mẫu có nhãn tương ứng
            label_indices = data[data[label_col] == label].index.tolist()
            
            # Xáo trộn các chỉ số
            np.random.shuffle(label_indices)
            
            # Tính số mẫu cho từng tập
            test_count = int(len(label_indices) * test_size)
            val_count = int(len(label_indices) * val_size)
            
            # Tách các chỉ số
            test_indices += label_indices[:test_count]
            val_indices += label_indices[test_count:test_count + val_count]
            train_indices += label_indices[test_count + val_count:]
        
        # Tạo các tập dữ liệu từ chỉ số đã chia
        train_data = data.loc[train_indices].reset_index(drop=True)
        val_data = data.loc[val_indices].reset_index(drop=True)
        test_data = data.loc[test_indices].reset_index(drop=True)
        
        # Lưu dữ liệu ra file CSV
        train_data.to_csv('../data/train_data.csv', index=False)
        val_data.to_csv('../data/val_data.csv', index=False)
        test_data.to_csv('../data/test_data.csv', index=False)
        
        return train_data, val_data, test_data

# Đọc dữ liệu từ file CSV
file_path = '../data/processed_data.csv' # File chứa dữ liệu đầu vào

try:
        df_tfidf_lsa = pd.read_csv(file_path)
except FileNotFoundError:
        print("File không tồn tại. Vui lòng kiểm tra đường dẫn.")
        exit()

    # Tách dữ liệu thành train, validation và test
train_df, val_df, test_df = stratified_train_val_test_split(
        data=df_tfidf_lsa,
        label_col='label',  
        test_size=0.15,     # 15% dữ liệu gốc cho tập test
        val_size=0.15,      # 15% dữ liệu gốc cho tập validation
        random_state=46
    )

In [146]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

df = pd.read_csv('../data/train_data.csv')

# Chuyển văn bản thành vector TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=4000)  
X_tfidf = tfidf_vectorizer.fit_transform(df['lyric'])

# Lưu TF-IDF vectorizer
tfidf_vectorizer_path = '../model/tfidf_vectorizer.pkl'
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
print(f"TF-IDF Vectorizer đã được lưu tại: {tfidf_vectorizer_path}")

# In thông tin TF-IDF
print("Shape of TF-IDF matrix:", X_tfidf.shape)

# Giảm chiều dữ liệu bằng LSA (SVD)
num_topics = 25  # Số lượng chủ đề muốn giữ lại
lsa = TruncatedSVD(n_components=num_topics, random_state=42)
X_lsa = lsa.fit_transform(X_tfidf)

# Lưu mô hình LSA
lsa_model_path = '../model/lsa_model.pkl'
joblib.dump(lsa, lsa_model_path)
print(f"LSA Model đã được lưu tại: {lsa_model_path}")

# In thông tin về các thành phần chính
print("Shape of LSA-transformed matrix:", X_lsa.shape)
print("Explained variance ratio:", lsa.explained_variance_ratio_.sum())

# Lưu TF-IDF và LSA vào DataFrame
df_tfidf_lsa = pd.DataFrame(X_lsa, columns=[f"Topic{i+1}" for i in range(num_topics)])
df_tfidf_lsa['label'] = df['label']

# Lưu dữ liệu đã xử lý vào file CSV
output_path = '../data/tfidf_lsa_train.csv'
df_tfidf_lsa.to_csv(output_path, index=False)
print(f"Dữ liệu TF-IDF và LSA đã được lưu tại: {output_path}")

TF-IDF Vectorizer đã được lưu tại: tfidf_vectorizer.pkl
Shape of TF-IDF matrix: (773, 4000)
LSA Model đã được lưu tại: lsa_model.pkl
Shape of LSA-transformed matrix: (773, 25)
Explained variance ratio: 0.19570743774975194
Dữ liệu TF-IDF và LSA đã được lưu tại: tfidf_lsa_train.csv


In [123]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score


data_train = pd.read_csv('../data/tfidf_lsa_train.csv')
data_val = pd.read_csv('../data/val_data.csv')


tfidf = joblib.load('../model/tfidf_vectorizer.pkl')
lsa = joblib.load('../model/lsa_model.pkl')
# Biến đổi tập validation
X_val_tfidf = tfidf.transform(data_val['lyric'])
X_val = lsa.transform(X_val_tfidf)

X_train = data_train.drop(columns=['label'])

y_train = data_train['label']
y_val = data_val['label']
# Thiết lập tham số cho GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2', None]
}

# Khởi tạo mô hình RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Thiết lập GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)

# Huấn luyện mô hình grid search
grid_search.fit(X_train, y_train)

# In kết quả tốt nhất
print("Best parameters found:", grid_search.best_params_)



Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Best parameters found: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


In [165]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib


data_train = pd.read_csv('../data/tfidf_lsa_train.csv')
data_val = pd.read_csv('../data/val_data.csv')


tfidf = joblib.load('../model/tfidf_vectorizer.pkl')
lsa = joblib.load('../model/lsa_model.pkl')
# Biến đổi tập validation
X_val_tfidf = tfidf.transform(data_val['lyric'])
X_val = lsa.transform(X_val_tfidf)

X_train = data_train.drop(columns=['label'])

y_train = data_train['label']
y_val = data_val['label']


# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(bootstrap= True, max_depth= None, min_samples_leaf= 2, min_samples_split= 2, n_estimators= 200, random_state= 53)

rf_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = rf_model.predict(X_val)

# Đánh giá mô hình
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Lưu mô hình nếu cần thiết
import joblib
model_path = '../model/random_forest_model.pkl'
joblib.dump(rf_model, model_path)
print(f"Mô hình đã được lưu tại: {model_path}")






Accuracy: 0.6890243902439024
Classification Report:
              precision    recall  f1-score   support

        buồn       0.68      0.81      0.74        62
         vui       0.71      0.76      0.74        46
  yêu thương       0.67      0.50      0.57        56

    accuracy                           0.69       164
   macro avg       0.69      0.69      0.68       164
weighted avg       0.69      0.69      0.68       164

Mô hình đã được lưu tại: random_forest_model.pkl


C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [166]:
import joblib
import pandas as pd

data_test = pd.read_csv('../data/test_data.csv')

tfidf = joblib.load('../model/tfidf_vectorizer.pkl')
lsa = joblib.load('../model/lsa_model.pkl')
# Biến đổi tập validation
X_test_tfidf = tfidf.transform(data_test['lyric'])
X_test = lsa.transform(X_test_tfidf)

y_test = data_test['label']

model_path = '../model/random_forest_model.pkl'
best_rf = joblib.load(model_path)
print("Mô hình đã được tải thành công.")


predicted_label = best_rf.predict(X_test)
accuracy_test = accuracy_score(predicted_label, y_test)
print("Accuracy for testing:", accuracy_test)
print("Classification Report:")
print(classification_report(y_test, predicted_label))



Mô hình đã được tải thành công.
Accuracy for testing: 0.6707317073170732
Classification Report:
              precision    recall  f1-score   support

        buồn       0.73      0.77      0.75        62
         vui       0.67      0.70      0.68        46
  yêu thương       0.60      0.54      0.57        56

    accuracy                           0.67       164
   macro avg       0.66      0.67      0.67       164
weighted avg       0.67      0.67      0.67       164



C:\Users\acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
